## Предсказание промышленного индекса Доу — Джонса по заголовкам новостей

Источник (соревнование Kaggle): https://www.kaggle.com/aaron7sun/stocknews

Входная коллекция данных состоит из двух частей:
1. заголовки новостных сообщений, входящих в топ-25 по популярности, опубликованных на  Reddit WorldNews Channel
2. бинаризованный промышленный индекса Доу — Джонса DJIA (0, если значение индекса за день понизилось, 1 – если осталось на прежнем уровне или выросло). 

Данные собраны за период с 2008-08-08 до 2016-07-01.

Данные представлены в файле Combined_News_DJIA.csv. Первая колонка – дата, вторая – бинаризованный промышленный индекса Доу — Джонса, оставшиеся 25 – заголовки новостей. 

### Часть 1 [3 балла] Предварительная обработка текстов
Проведите предобработку текстов: если считаете нужным, выполните токенизацию, приведение к нижнему регистру, лемматизацию и/или стемминг. 
Ответьте на следующие вопросы:
1. Есть ли корреляция между средней длинной текста за день и DJIA?
2. Есть ли корреляция между количеством упоминаний Барака Обамы и США в день и DJIA? Учтите разные варианты написания США.
3. Каких статей больше: статей о России и Путине или об Исламском государстве (запрещенной законом РФ террористическая организации)?
4. О каких кризисах (crisis) пишут статьи?


### Часть 2 [5 баллов] Классификация

Вам предстоит решить следующую задачу: по текстам новостей за день определить, вырастет или понизится DJIA. То есть, метки класса (y) заданы DJIA, признаки (X) требуется извлечь из текстов.

Обучающее и тестовое множество строится так: данные до начала 2015 года используются для обучения, данные с 2015 года и позже – для тестировани.

Подсказка:

Используйте любой известный вам алгоритм классификации текстов для того,  Используйте $tf-idf$ преобразование, сингулярное разложение, нормировку признакого пространства и любые другие техники обработки данных, которые вы считаете нужным. Используйте accuracy и F-measure  для оценки качества классификации. Покажите, как  $tf-idf$ преобразование или сингулярное разложение или любая другая использованная вами техника влияет на качество классификации. 

Если у выбранного вами алгоритма есть гиперпараметры (например, $\alpha$ в преобразовании Лапласа для метода наивного Байеса), покажите, как изменение гиперпараметра влияет на качество классификации. 

### Часть 3 [2 балла] Творческая
Придумайте и попытайтесь сделать еще что-нибудь, чтобы улучшить качество классификации. Например:
* использовать в качестве признаков только именованные сущности; 
* использовать в качестве признаков скрытые темы;
* добавить признак, отвечающий за какие-то важные темы или тональность новостей.

# Домашка 

In [1]:
from pandas import DataFrame, concat, options
from nltk.corpus import stopwords
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import TruncatedSVD

# import torch
# from torch import nn
# import torch.nn.functional as F
# from skorch.net import NeuralNetClassifier

stop = stopwords.words('english')
options.mode.chained_assignment = None

Загружаем данные

In [2]:
data = DataFrame.from_csv('stocknews/Combined_News_DJIA.csv').reset_index().fillna(' ')

In [3]:
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']
y_train = train.Label.values
y_test = test.Label.values

In [4]:
col_number = 26

### Тут будет анализ данных

### Простые фичи

In [5]:
lambda_func_features = [
    (lambda x: len(str(x).split()), 'NumWords'),
    (lambda x: len(set(str(x).split())), 'NumUniqueWords'),
    (lambda x: len(str(x)), 'NumChars'),
    (lambda x: len([w for w in str(x).lower().split() if w in stop]), 'NumStopWords'),
    (lambda x: np.mean([len(w) for w in str(x).split()]), 'MeanWordLen')
]

In [6]:
def generate_features(train, test, lambda_func, func_name):
    train_features = DataFrame([train.loc[:,'Top' + str(col)].apply(lambda_func) for col in range(1, col_number)]).transpose()
    test_features = DataFrame([test.loc[:,'Top' + str(col)].apply(lambda_func) for col in range(1, col_number)]).transpose()
    train_features.columns = [func_name + str(i) for i in range(1, col_number)]
    test_features.columns = [func_name + str(i) for i in range(1, col_number)]
    return concat([train, train_features], axis=1), concat([test, test_features], axis=1)

In [7]:
for lambda_func, func_name in lambda_func_features:
    train, test = generate_features(train, test, lambda_func, func_name)

/Users/defeater/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


### Фичи на основе предсказаний классификаторов на векторизаторах

In [8]:
def make_raw_text_col(df):
    df['text'] = df['Top1'].str[1:] + ' '
    for i in df.loc[:,'Top2':'Top25']:
        df['text'] += df[i].str[1:] + ' '
    df['text'] = df['text'].str.lower().str.replace('[^a-zA-Z ]', '')
    return df
    
train = make_raw_text_col(train)
test = make_raw_text_col(test)

In [9]:
vectorizers = [
    (CountVectorizer(stop_words='english', ngram_range=(1,3)), 'CountVectorizer'),
    (TfidfVectorizer(ngram_range=(1,5), analyzer='char'), 'TfIdfVectorizerChar')
]

In [10]:
kfolds = 5

for vec, vec_name in vectorizers:
    vectorizer = vec
    full = vectorizer.fit_transform(np.hstack((train.text.values, test.text.values)))
    X_train = vectorizer.transform(train.text.values)
    X_test = vectorizer.transform(test.text.values)

    pred_full_test = 0
    pred_train = np.zeros([train.shape[0], len(train.Label.unique())])

    for dev_index, val_index in KFold(n_splits=kfolds, shuffle=True, random_state=42).split(train):
        dev_X, val_X = X_train[dev_index], X_train[val_index]
        dev_y, val_y = y_train[dev_index], y_train[val_index]
        model = MultinomialNB()
        model.fit(dev_X, dev_y)
        pred_full_test = pred_full_test + model.predict_proba(X_test)
        pred_train[val_index,:] = model.predict_proba(val_X)

    pred_full_test = pred_full_test / kfolds

    train[vec_name + 'Zero'] = pred_train[:,0]
    train[vec_name + 'One'] = pred_train[:,1]
    test[vec_name + 'Zero'] = pred_full_test[:,0]
    test[vec_name + 'One'] = pred_full_test[:,1]

### Фичи на основе компонент векторов

In [11]:
svd_components = 20

vectorizer = TfidfVectorizer(ngram_range=(1,5), analyzer='char')
full = vectorizer.fit_transform(np.hstack((train.text.values, test.text.values)))
X_train = vectorizer.transform(train.text.values)
X_test = vectorizer.transform(test.text.values)

svd = TruncatedSVD(n_components=svd_components, algorithm='arpack')
svd.fit(full)
train_svd = DataFrame(svd.transform(X_train))
test_svd = DataFrame(svd.transform(X_test))
    
train_svd.columns = ['SVD' + str(i) for i in range(svd_components)]
test_svd.columns = ['SVD' + str(i) for i in range(svd_components)]

train = concat([train, train_svd], axis=1, join_axes=[train.index])
test = concat([test, test_svd], axis=1, join_axes=[test.index])

### Фичи сгенерированы, убираем лишнее

In [12]:
train = train.drop(train.columns[0:col_number+2], axis=1)
test = test.drop(test.columns[0:col_number+2], axis=1)
# train = train.drop('text', axis=1)
# test = test.drop('text', axis=1)

### Задаём параметры градиентного бустинга

In [13]:
params = {}
params['objective'] = 'multi:softprob'
params['eta'] = 0.1
params['max_depth'] = 3
params['silent'] = 1
params['num_class'] = 3
params['eval_metric'] = 'mlogloss'
params['min_child_weight'] = 1
params['subsample'] = 0.8
params['colsample_bytree'] = 0.3
params['seed'] = 0

### Классификация

In [43]:
boost_rounds = 20
kfolds = 5
pred_full_test = 0
pred_train = np.zeros([train.shape[0], len(set(y_train))])
for dev_index, val_index in KFold(n_splits=kfolds, shuffle=True, random_state=42).split(train):
    dev_X, val_X = train.loc[dev_index], train.loc[val_index]
    dev_y, val_y = y_train[dev_index], y_train[val_index]
    xgtrain = xgb.DMatrix(dev_X, dev_y)
    xgtest = xgb.DMatrix(test)
    model = xgb.train(params=list(params.items()), dtrain=xgtrain, num_boost_round=boost_rounds)
    predictions = model.predict(xgtest, ntree_limit=model.best_ntree_limit)
    pred_full_test = pred_full_test + predictions
pred_full_test = pred_full_test / kfolds

### Считаем результаты

Текстовые фичи

In [11]:
f1_score(pred_full_test.argmax(axis=1), y_test) 

0.67034990791896865

In [12]:
accuracy_score(pred_full_test.argmax(axis=1), y_test) 

0.52645502645502651

Фичи с вероятностями на векторизации

In [43]:
f1_score(pred_full_test.argmax(axis=1), y_test) 

0.65461121157323687

In [44]:
accuracy_score(pred_full_test.argmax(axis=1), y_test) 

0.49470899470899471

Фичи на компонентах векторов

In [93]:
f1_score(pred_full_test.argmax(axis=1), y_test) 

/Users/defeater/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


0.0

In [94]:
accuracy_score(pred_full_test.argmax(axis=1), y_test) 

0.49206349206349204

Всё вместе

In [36]:
f1_score(pred_full_test.argmax(axis=1), y_test) 

0.66019417475728159

In [37]:
accuracy_score(pred_full_test.argmax(axis=1), y_test) 

0.53703703703703709